In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [0]:
n_hidden = 35
# 순환 신경망의 노드수 = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"

# 예시에 사용할 문장은 string에 사용할 문자, 기호들은 chars 에 저장
char_list = [i for i in chars]
# chars 의 요소들을 리스트로 만들어줌 
n_letters = len(char_list)

In [0]:
# 문자를 그대로 쓰지않고 one-hot 벡터로 바꿔서 연산에 쓰도록 하겠습니다.

#Start = [0 0 0 … 1 0]
#a =     [1 0 0 … 0 0]
#b =     [0 1 0 … 0 0]
#c =     [0 0 1 … 0 0]
#...
#end =   [0 0 0 … 0 1]

In [0]:
def string_to_onehot(string):
  # 문자를 one_hot으로 만드는 함수
  start = np.zeros(shape=len(char_list),dtype=int)
  end = np.zeros(shape=len(char_list), dtype=int)
  start[-2] = 1
  # start 맨 뒤에서 두번째는 1
  end[-1] = 1
  # end 맨 뒤에는 1
  for i in string:
    idx = char_list.index(i)
    zero = np.zeros(shape=n_letters,dtype=int)
    # zero 는 list의 길이의 0 배열 
    zero[idx] = 1
    # 해당 인덱스에 1 넣고 나머지는 다 0

    start = np.vstack([start,zero])
    # start 에 [start,zero] 의 배열을 추가함 
  output = np.vstack([start,end])
  # output 에 [start,end]의 배열을 추가함
  return output

In [0]:
def one_hot_to_word(onehot_1):
  # one_hot을 다시 문자로 만드는 함수
  onehot = torch.Tensor.numpy(onehot_1)
  # 토치 텐서를 입력으로 받아서 이를 넘파이 배열로 변환
  # onehot의 argmax를 찾아서 다시 char_list로 반환
  return char_list[onehot.argmax()]

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RNN, self).__init__()
    # input_size, output_size 는 배열의 길이
    # hidden_size = 35
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size

    # 단어 하나를 입력값으로 받고 은닉층 하나를 통과시켜 결괏값을 내는 구조
    self.i2h = nn.Linear(input_size,hidden_size)
    self.h2h = nn.Linear(hidden_size,hidden_size)
    self.i2o = nn.Linear(hidden_size,output_size)
    self.act_fn = nn.Tanh()
    # 활성화 함수는 Tanh 

  def forward(self,input,hidden):
    hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
    output = self.i2o(hidden)
    return output, hidden

  def init_hidden(self):
    # 이전 시간의 은닉층 연산값이 없는 초기의 은닉층 값은 0으로 초기화
    return torch.zeros(1,self.hidden_size)

  
rnn = RNN(n_letters,n_hidden,n_letters)






In [0]:
loss_func = nn.MSELoss()
# 손실함수 MSE
optimizer = torch.optim.Adam(rnn.parameters(),lr=lr)

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())
# 학습하고자 했던 문장을 원-핫 벡터로 변환한 넘파이 배열을 토치 텐서형태로 형 변환
# 앞서 만든 함수대로 start_token + 문장 + end_token 형태의 행렬로 생성 
# 학습 부분.. 문장 전체를 학습하는 과정은 epochs 에 지정한 만큼 반복한다 
for i in range(epochs):
  rnn.zero_grad()
  total_loss = 0
  hidden = rnn.init_hidden()

  for j in range(one_hot.size()[0]-1):
    # 원-핫 벡터는 문장에 있는 단어 순서대로 배열되어 있기 때문에 j번째 인덱스에 해당하는 값이 입력으로
    # 들어오면 j+1 해당하는 값이 target이 된다.
    input_ = one_hot[j:j+1,:]
    target = one_hot[j+1]

    output,hidden = rnn.forward(input_,hidden)
    # forward 를 통해 input_ , hidden 을 output, hidden 으로 전달
    loss = loss_func(output.view(-1),target.view(-1))
    # 손실함수는 view 를 거쳐 이쁘게 들어감
    total_loss += loss
    input_ = output

  total_loss.backward()
  optimizer.step()

  if i % 10 ==0:
    print(total_loss)

tensor(0.0762, grad_fn=<AddBackward0>)
tensor(0.2071, grad_fn=<AddBackward0>)
tensor(0.1342, grad_fn=<AddBackward0>)
tensor(0.1019, grad_fn=<AddBackward0>)
tensor(0.0836, grad_fn=<AddBackward0>)
tensor(0.0720, grad_fn=<AddBackward0>)
tensor(0.0632, grad_fn=<AddBackward0>)
tensor(0.0561, grad_fn=<AddBackward0>)
tensor(0.0501, grad_fn=<AddBackward0>)
tensor(0.0600, grad_fn=<AddBackward0>)
tensor(0.0444, grad_fn=<AddBackward0>)
tensor(0.0390, grad_fn=<AddBackward0>)
tensor(0.0347, grad_fn=<AddBackward0>)
tensor(0.0314, grad_fn=<AddBackward0>)
tensor(0.0286, grad_fn=<AddBackward0>)
tensor(0.0261, grad_fn=<AddBackward0>)
tensor(0.0469, grad_fn=<AddBackward0>)
tensor(0.0279, grad_fn=<AddBackward0>)
tensor(0.0233, grad_fn=<AddBackward0>)
tensor(0.0210, grad_fn=<AddBackward0>)
tensor(0.0191, grad_fn=<AddBackward0>)
tensor(0.0249, grad_fn=<AddBackward0>)
tensor(0.0203, grad_fn=<AddBackward0>)
tensor(0.0190, grad_fn=<AddBackward0>)
tensor(0.0159, grad_fn=<AddBackward0>)
tensor(0.0145, grad_fn=<A

In [0]:
start = torch.zeros(1,len(char_list))
# 1행 5열짜리의 zeros 배열을 만들어줌!
print(start)
start[:,-2] = 1
# 모든 행의 마지막 두번째 인덱스에 1 을 넣어줌 

# 기울기 없이 테스트 한다 
with torch.no_grad():
  hidden = rnn.init_hidden()
  input_ = start
  output_string = ""
  for i in range(len(string)):
    output,hidden = rnn.forward(input_,hidden)
    # 
    output_string += one_hot_to_word(output.data)
    input_ = output
  
print(output_string)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
hello  rem bwem hwem b eemnwe. pano r.m hwem nbn
